## Read and Clean data

In [97]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

sales = pd.read_csv("data.csv")

def sanitize_numbers(df):
    df.str.replace('-', '')
    return pd.to_numeric(df.str.replace('Rs.||\s|€|-','', regex=True).str.replace(',', '.'))


sales = sales.rename(columns={
    "Price = competitor analys price ": "Price",
    "Product Name": "Name",
    "Product ID": "Id",
})
sales["Price"] = sanitize_numbers(sales["Price"])
sales["Sales"] = sanitize_numbers(sales["Sales"])
sales["Sales Last Year"] = sanitize_numbers(sales["Sales Last Year"])
sales["Views"] = sanitize_numbers(sales["Views"])
sales["Views Last Year"] = sanitize_numbers(sales["Views Last Year"])
sales["Quantity"] = sanitize_numbers(sales["Quantity"])
sales["Quantity Last Year"] = sanitize_numbers(sales["Quantity Last Year"])
sales["Name"] = sales["Name"].astype("string")

sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 479 entries, 0 to 478
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  479 non-null    int64  
 1   Name                479 non-null    string 
 2   Price               479 non-null    float64
 3   Sales               263 non-null    float64
 4   Sales Last Year     175 non-null    float64
 5   Views               429 non-null    float64
 6   Views Last Year     252 non-null    float64
 7   Quantity            263 non-null    float64
 8   Quantity Last Year  175 non-null    float64
dtypes: float64(7), int64(1), string(1)
memory usage: 33.8 KB


## Basic description

In [96]:
sales.head()

,Id,Name,Price,Sales,Sales Last Year,Views,Views Last Year,Quantity,Quantity Last Year
0,1257666,Vol en Montgolfière en France pour un,149.9,227698.10,130537.92,56225.0,61171.0,1519.0,871.0
1,1270458,Vol en montgolfière avec dégustation et visite...,389.9,218344.00,67907.58,13500.0,6566.0,560.0,174.0
2,1259281,Vol en Montgolfière en semaine pour deux,289.9,167079.03,113544.17,19928.0,26616.0,576.0,392.0
3,1458497,Vol en montgolfi?re d?1h au-dessus des ch?teau...,289.9,113302.58,NaN,8388.0,NaN,391.0,NaN
4,1458485,Vol en montgolfi?re d'1h en duo au-dessus des ...,359.9,63822.27,NaN,3829.0,NaN,177.0,NaN
